In [ ]:
# default_exp data_reader

In [3]:
import gcp_runner.core
gcp_runner.core.export_and_reload_all(silent=True)

In [4]:
#export
from criteo_nbdev.constants import *

def get_dataset_size(dataset_size: DATASET_SIZE_TYPE, dataset_type: DATASET_TYPE):
    if dataset_size == DATASET_SIZE_TYPE.full:
        if dataset_type == DATASET_TYPE.train:
            return FULL_TRAIN_DATASET_SIZE
        else:
            return FULL_TEST_DATASET_SIZE
    else:
        if dataset_type == DATASET_TYPE.train:
            return SMALL_TRAIN_DATASET_SIZE
        else:
            return SMALL_TEST_DATASET_SIZE
        
def get_steps_per_epoch(dataset_size: DATASET_SIZE_TYPE, dataset_type: DATASET_TYPE):
    return get_dataset_size(dataset_size, dataset_type) // BATCH_SIZE

def get_max_steps():
    global EPOCHS
    return EPOCHS * get_training_steps_per_epoch()

In [5]:
#export
from google.cloud import bigquery
from criteo_nbdev.constants import *

def get_mean_and_std_dicts():
    client = bigquery.Client(project=PROJECT_ID)
    query = """
    select
    AVG(int1) as avg_int1, STDDEV(int1) as std_int1,
    AVG(int2) as avg_int2, STDDEV(int2) as std_int2,
    AVG(int3) as avg_int3, STDDEV(int3) as std_int3,
    AVG(int4) as avg_int4, STDDEV(int4) as std_int4,
    AVG(int5) as avg_int5, STDDEV(int5) as std_int5,
    AVG(int6) as avg_int6, STDDEV(int6) as std_int6,
    AVG(int7) as avg_int7, STDDEV(int7) as std_int7,
    AVG(int8) as avg_int8, STDDEV(int8) as std_int8,
    AVG(int9) as avg_int9, STDDEV(int9) as std_int9,
    AVG(int10) as avg_int10, STDDEV(int10) as std_int10,
    AVG(int11) as avg_int11, STDDEV(int11) as std_int11,
    AVG(int12) as avg_int12, STDDEV(int12) as std_int12,
    AVG(int13) as avg_int13, STDDEV(int13) as std_int13
    from `{project}.{dataset}.{table_name}`
  """.format(
        project=PROJECT_ID,
        dataset=DATASET_ID,
        table_name='days',
    )
    query_job = client.query(
        query,
        location=LOCATION,
    )  # API request - starts the query

    df = query_job.to_dataframe()

    mean_dict = dict((field[0].replace('avg_', ''), df[field[0]][0])
                     for field in df.items() if field[0].startswith('avg'))
    std_dict = dict((field[0].replace('std_', ''), df[field[0]][0])
                    for field in df.items() if field[0].startswith('std'))
    return (mean_dict, std_dict)

In [6]:
from criteo_nbdev.core import skip; skip()
get_mean_and_std_dicts()

In [ ]:
#export
from google.cloud import bigquery
from criteo_nbdev.constants import *

def get_vocabulary_size_dict():
    client = bigquery.Client(project=PROJECT_ID)
    query = """
    SELECT
    COUNT(DISTINCT cat1) as cat1,
    COUNT(DISTINCT cat2) as cat2,
    COUNT(DISTINCT cat3) as cat3,
    COUNT(DISTINCT cat4) as cat4,
    COUNT(DISTINCT cat5) as cat5,
    COUNT(DISTINCT cat6) as cat6,
    COUNT(DISTINCT cat7) as cat7,
    COUNT(DISTINCT cat8) as cat8,
    COUNT(DISTINCT cat9) as cat9,
    COUNT(DISTINCT cat10) as cat10,
    COUNT(DISTINCT cat11) as cat11,
    COUNT(DISTINCT cat12) as cat12,
    COUNT(DISTINCT cat13) as cat13,
    COUNT(DISTINCT cat14) as cat14,
    COUNT(DISTINCT cat15) as cat15,
    COUNT(DISTINCT cat16) as cat16,
    COUNT(DISTINCT cat17) as cat17,
    COUNT(DISTINCT cat18) as cat18,
    COUNT(DISTINCT cat19) as cat19,
    COUNT(DISTINCT cat20) as cat20,
    COUNT(DISTINCT cat21) as cat21,
    COUNT(DISTINCT cat22) as cat22,
    COUNT(DISTINCT cat23) as cat23,
    COUNT(DISTINCT cat24) as cat24,
    COUNT(DISTINCT cat25) as cat25,
    COUNT(DISTINCT cat26) as cat26
    FROM
      from `{project}.{dataset}.{table_name}`
  """.format(
        project=PROJECT_ID,
        dataset=DATASET_ID,
        table_name='days',
    )
    query_job = client.query(
        query,
        location=LOCATION,
    )  # API request - starts the query

    df = query_job.to_dataframe()
    dictionary = dict((field[0], df[field[0]][0]) for field in df.items())
    return dictionary


In [7]:
from criteo_nbdev.core import skip; skip()
get_vocabulary_size_dict()

In [8]:
#export
from google.cloud import bigquery
from criteo_nbdev.constants import *
import tensorflow as tf

#TODO: add step to generate corpus and update script to use right table

def get_corpus_dict():
    table_name = 'days' if DATASET_SIZE == DATASET_SIZE_TYPE.full else 'small'
    client = bigquery.Client(project=PROJECT_ID)
    query = """
    select
    cat_name,
    cat_value,
    cat_index
    from `alekseyv-scalableai-dev.criteo_kaggle.{table_name}_corpus`
  """.format(table_name=table_name)
    query_job = client.query(
        query,
        location="US",
    )  # API request - starts the query

    df = query_job.to_dataframe()
    corpus = dict()
    for _, row in df.iterrows():
        cat_name = row[0]
        cat_value = row[1]
        cat_index = row[2]
        if not cat_name in corpus:
            corpus[cat_name] = dict()
        if cat_value is None:
            cat_value = ''
        corpus[cat_name][cat_value] = cat_index
    return corpus

def corpus_to_lookuptable(corpus):
    lookup_dict = dict()
    for key, value in corpus.items():
        initializer = tf.lookup.KeyValueTensorInitializer(
            list(value.keys()),
            list(value.values()),
            key_dtype=tf.string,
            value_dtype=tf.int64)
        # cat_index in corpus starts with 1, reserving 0 for out of vocabulary values
        lookup_table = tf.lookup.StaticHashTable(initializer, 0)
        lookup_dict[key] = lookup_table
    return lookup_dict


def get_corpus(embeddings_mode):
    if embeddings_mode == EMBEDDINGS_MODE_TYPE.manual or embeddings_mode == EMBEDDINGS_MODE_TYPE.vocabular:
        return corpus_to_lookuptable(get_corpus_dict())
    else:
        return dict()

In [9]:
from criteo_nbdev.core import skip; skip()
get_corpus(EMBEDDINGS_MODE_TYPE.manual)

{'cat1': <tensorflow.python.ops.lookup_ops.StaticHashTable at 0x7f98503684e0>,
 'cat2': <tensorflow.python.ops.lookup_ops.StaticHashTable at 0x7f9824bffb00>,
 'cat3': <tensorflow.python.ops.lookup_ops.StaticHashTable at 0x7f9824bff940>,
 'cat4': <tensorflow.python.ops.lookup_ops.StaticHashTable at 0x7f9824bff780>,
 'cat5': <tensorflow.python.ops.lookup_ops.StaticHashTable at 0x7f9824bff5c0>,
 'cat6': <tensorflow.python.ops.lookup_ops.StaticHashTable at 0x7f9824bff400>,
 'cat7': <tensorflow.python.ops.lookup_ops.StaticHashTable at 0x7f9824bff240>,
 'cat8': <tensorflow.python.ops.lookup_ops.StaticHashTable at 0x7f9824bff048>,
 'cat9': <tensorflow.python.ops.lookup_ops.StaticHashTable at 0x7f9824bff208>,
 'cat10': <tensorflow.python.ops.lookup_ops.StaticHashTable at 0x7f9824bff3c8>,
 'cat11': <tensorflow.python.ops.lookup_ops.StaticHashTable at 0x7f9824bff588>,
 'cat12': <tensorflow.python.ops.lookup_ops.StaticHashTable at 0x7f9824bff748>,
 'cat13': <tensorflow.python.ops.lookup_ops.Stati

In [17]:
#export
from criteo_nbdev.constants import *
from google.cloud import bigquery

import tensorflow as tf
from tensorflow.python.framework import dtypes
from tensorflow.python.data.experimental.ops import interleave_ops
from tensorflow.python.data.ops import dataset_ops

from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession

import gcp_runner.core
gcp_runner.core.export_and_reload_all(silent=True)

# @tf.function is not really necessary here since function is used in tf.data pipeline
# Use print(tf.autograph.to_code(transform_row.python_function)) to see code generated by AutoGraph
@tf.function
def transform_row(row_dict, mean_dict, std_dict, corpus, embeddings_mode:EMBEDDINGS_MODE_TYPE):
    dict_without_label = dict(row_dict)
    label = dict_without_label.pop('label')
    for field in CSV_SCHEMA:
        if (field.name.startswith('int')):
            # use normalized mean value if data is missing (value will be 0)
            value = float(dict_without_label[field.name])
            dict_without_label[field.name] = (
                value - mean_dict[field.name]) / std_dict[field.name]
        elif field.name.startswith('cat'):
            if embeddings_mode == EMBEDDINGS_MODE_TYPE.none:
                dict_without_label.pop(field.name)
            elif embeddings_mode == EMBEDDINGS_MODE_TYPE.manual:
                cat = dict_without_label[field.name]
                if cat is None:
                    cat = ''
                cat_index = corpus[field.name].lookup(cat)
                if cat_index is None:
                    tf.print('not found for {}'.format(field.name))
                    cat_index = tf.constant(-1)
                dict_without_label[field.name] = cat_index
    return (dict_without_label, label)

def get_bigquery_table_name(dataset_size: DATASET_SIZE_TYPE, dataset_type: DATASET_TYPE):
    return dataset_type.name + '_' + dataset_size.name

def read_bigquery(dataset_size: DATASET_SIZE_TYPE, dataset_type: DATASET_TYPE, embedding_mode: EMBEDDINGS_MODE_TYPE):
    table_name = get_bigquery_table_name(dataset_size, dataset_type)
    (mean_dict, std_dict) = get_mean_and_std_dicts()
    corpus = get_corpus(embedding_mode)
    requested_streams_count = 10
    tensorflow_io_bigquery_client = BigQueryClient()
    read_session = tensorflow_io_bigquery_client.read_session(
        "projects/" + PROJECT_ID,
        PROJECT_ID, table_name, DATASET_ID,
        list(field.name for field in CSV_SCHEMA),
        list(dtypes.int64 if field.field_type == 'INTEGER'
             else dtypes.string for field in CSV_SCHEMA),
        requested_streams=requested_streams_count)

    # manually sharding output instead of using return read_session.parallel_read_rows()
    streams = read_session.get_streams()
    # streams_count = len(streams) # does not work for Estimator
    streams_count = tf.size(streams)
    streams_count64 = tf.cast(streams_count, dtype=tf.int64)
    streams_ds = dataset_ops.Dataset.from_tensor_slices(
        streams).shuffle(buffer_size=streams_count64)
    dataset = streams_ds.interleave(
        read_session.read_rows,
        cycle_length=streams_count64,
        num_parallel_calls=streams_count64)

    def transform_row_function(row): return transform_row(
        row, mean_dict, std_dict, corpus, embedding_mode)

    transformed_ds = dataset\
        .batch(BATCH_SIZE) \
        .shuffle(50) \
        .map(transform_row_function, num_parallel_calls=streams_count) \
        .prefetch(50)

    # TODO: enable once tf.data.experimental.AutoShardPolicy.OFF is available
    # Interleave dataset is not shardable, turning off sharding
    # See https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras#dataset_sharding_and_batch_size
    # Instead we are shuffling data.
    # options = tf.data.Options()
    #  options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
    # return transformed_ds.with_options(options)
    return transformed_ds


In [ ]:
from criteo_nbdev.core import skip; skip()

#BATCH_SIZE = 4
dataset = read_bigquery(DATASET_SIZE_TYPE.small, DATASET_TYPE.validation, EMBEDDINGS_MODE_TYPE.hashbucket)
for row in dataset.take(2):
    print(row)

In [ ]:


def transofrom_row_gcs(row_tuple, mean_dict, std_dict, corpus):
    row_dict = dict(
        zip(list(field.name for field in CSV_SCHEMA) + ['row_hash'], list(row_tuple)))
    row_dict.pop('row_hash')
    return transform_row(row_dict, mean_dict, std_dict, corpus)


def _get_file_names(file_pattern):
    if isinstance(file_pattern, list):
        if not file_pattern:
            raise ValueError("File pattern is empty.")
        file_names = []
        for entry in file_pattern:
            file_names.extend(gfile.Glob(entry))
    else:
        file_names = list(gfile.Glob(file_pattern))

    if not file_names:
        raise ValueError("No files match %s." % file_pattern)
    return file_names


def read_gcs(table_name):
    if DATASET_SIZE == DATASET_SIZE_TYPE.small:
        table_name += '_small'
    else:
        table_name += '_full'

    gcs_filename_glob = 'gs://alekseyv-scalableai-dev-public-bucket/criteo_kaggle_from_bq/{}*'.format(
        table_name)
    file_names = _get_file_names(gcs_filename_glob)
    num_parallel_calls = max(10, len(file_names))
    file_names_ds = dataset_ops.Dataset.from_tensor_slices(
        file_names).shuffle(buffer_size=20)
    record_defaults = list(tf.int32 if field.name == 'label' else tf.constant(0, dtype=tf.int32) if field.name.startswith(
        'int') else tf.constant('', dtype=tf.string) for field in CSV_SCHEMA) + [tf.string]
    dataset = file_names_ds.interleave(
        lambda file_name: tf.data.experimental.CsvDataset(
            file_name, record_defaults, field_delim='\t', header=False),
        cycle_length=num_parallel_calls,
        num_parallel_calls=num_parallel_calls)

    corpus = get_corpus()
    (mean_dict, std_dict) = get_mean_and_std_dicts()
    transofrom_row_gcs_function = lambda *row_tuple: transofrom_row_gcs(
        row_tuple, mean_dict, std_dict, corpus)

    transformed_ds = dataset\
        .batch(BATCH_SIZE) \
        .shuffle(50) \
        .map(transofrom_row_gcs_function, num_parallel_calls=num_parallel_calls) \
        .prefetch(50)
    return transformed_ds


In [ ]:

from criteo_nbdev.constants import *

def get_dataset(dataset_source: DATASET_SOURCE_TYPE, 
                dataset_size: DATASET_SIZE_TYPE, 
                dataset_type: DATASET_TYPE):
    if DATASET_SOURCE == DATASET_SOURCE_TYPE.gcs:
        return read_gcs(dataset_size, dataset_type)
    else:
        return read_bigquery(dataset_size, dataset_type)